# aritmética.py

[TOC] 

anyway

[[TOC]]

[Versión en GitHub](https://github.com/profedrini/euler/blob/master/aritmetica.py)

Muchos problemas del proyecto Euler involucran funciones aritméticas (determinar si un número es primo, encontrar su factorización,  calcular el número de divisores, etcétera).

Por ello, implemento una librería para funciones aritméticas.

* Principios: En caso de ser posible, preferir usar ram/memoria a ciclos de cpu.
* Corolario: Almacenar valores calculados para evitar calcularlos de nuevo.

El objetivo de este documento es discutir algunas de las funciones que se encuentran en dicho archivo.

En lo posible haremos uso de las siguientes librerías estándar

In [1]:
import itertools as it
import math

Como base de todo nuestro proceso, generamos una lista con los números primos hasta cierto valor. Aunque en la versión final de la librería, simplemente cargamos la lista directamente en código, revisamos de todos modos algunas formas de hacerlo. 

Por ejemplo, si queremos calcular todos los números primos menores que $10^4$:

* Generamos una lista  [0,1,2,3, ..., 9999] e iremos cambiando los valores que no sean primos por cero. 
* Dado que 1 es un caso especial, lo descartamos manualmente.
* Posteriormente, para cada $d\le \sqrt{MAXP}$, si L[d]=0, ya hemos determinado que no es primo (y por tanto todos sus múltiplos también). En caso contrario, tenemos un primo y usamos la criba para hacer cero todos sus múltiplos en la lista.
  * Esto lo hacemos con itertools.islice, que selecciona los índices 2d, 3d, 4d, ....
* Al final, filtramos todos los valores que no se hayan anulado.

In [2]:
%%timeit
MAXP=10**4
L=[n for n in range(MAXP)]  
L[0]=0
L[1]=0
for d in range(2,int(math.sqrt(MAXP))+1):
    if L[d]>0:   # Si ya descartamos d, ya descartamos a sus múltiplos, sólo se necesita verificar si L[d]=True
        for k in it.islice(range(MAXP), 2*d, None, d): L[k]=0
    #if d*d>MAXP: break
Primos=[P for P in L if P>0 ]
#print(Primos)

100 loops, best of 3: 5.92 ms per loop


Usando el mismo código para hallar todos los primos menores que un millón:

In [3]:
%%time
MAXP=10**6
L=[n for n in range(MAXP)]  
L[0]=0
L[1]=0
for d in range(2,int(math.sqrt(MAXP))+1):
    if L[d]>0:   # Si ya descartamos d, ya descartamos a sus múltiplos, sólo se necesita verificar si L[d]=True
        for k in it.islice(range(MAXP), 2*d, None, d): L[k]=0
ListaPrimos=[P for P in L if P>0 ]

CPU times: user 3.47 s, sys: 21 ms, total: 3.49 s
Wall time: 3.49 s


Toma menos de 10 segundos el hallar todos los primos menores que un millón.

## Alternativas
Sólo por curiosidad, haremos un intento híbrido: generamos todos los primos menores que $10^3$ y luego los usamos para cribar los números hasta $10^3$:

In [4]:
%%time
MAXP=10**6
MEDP=10**3
S=[n for n in range(MEDP)]   # En python3, range es un generador, no existe xrange
S[0]=0
S[1]=0
for d in range(2,int(math.sqrt(MEDP))+1):
    if S[d]>0:   # Si ya descartamos d, también sus múltiplos, sólo hay que verificar si L[d]=True
        for k in it.islice(range(MEDP), 2*d, None, d): S[k]=0
S=[p for p in S if p>0 ]
L=[n for n in range(MEDP, MAXP)]
for p in S:
    L = [m for m in L if m%p>0]
    
ListaPrimos=(S+L)

CPU times: user 3.53 s, sys: 3 ms, total: 3.53 s
Wall time: 3.53 s


Una tercera posibilidad consiste en ir eliminando de la lista los números que no sean primos, en vez de hacer que sus índices sean iguales a cero y recolectar al final:

In [5]:
%%time
MAXP=10**6
L=[2]+[n for n in range(3,MAXP,2)]  
for d in range(3,int(math.sqrt(MAXP))+1,2):
    if d in L:   # Si ya eliminamos d, ya eliminamos a sus múltiplos, sólo se necesita verificar si L[d]=True
        L = [m for m in L if m<=d or m%d>0]
ListaPrimos=L

CPU times: user 5.44 s, sys: 4 ms, total: 5.45 s
Wall time: 5.44 s


La razón por la cual esto es más tardado, probablemente sea que se están copiando y modificando listas, y el manejo de memoria correspondiente lleve más tiempo que simplemente cambiar los valores y modificar listas sólo una vez al final.

Dado que no hay una mejora sustancial en las dos versiones, optamos por quedarnos con la versión primera, ya que es más simple.

Dado que la verificación de pertenencia es más rápida con un conjunto, creamos paralelamente la versión no ordenada:

Finalmente, envolvemos todo en una bonita función

In [ ]:
#aritmetica.py

import itertools as it
import math

# Inicializa una lista de primos para las demás funciones
L=[n for n in range(MAXP)]  
L[0]=0
L[1]=0
for d in range(2,int(math.sqrt(MAXP))+1):
    if L[d]>0: for k in it.islice(range(MAXP), 2*d, None, d): L[k]=0


ListaPrimos=[P for P in L if P>0 ]
ConjuntoPrimos=set(ListaPrimos)  # Para acelerar búsqueda cuando no nos importa el orden.